# Les closures

---

- Il existe en Rust un type de fonction un peu spécial, qui se nomme closure. Contrairement aux fonctions, elles peuvent être `anonymes`, les paramètres sont notes entre `| |` plutôt qu'entre parenthèses `( )` et elles ont surtout la capacité de capturer les variables de leur environnement (contrairement aux fonctions). La capture se fait soit par `référence &`, soit par `référence mutable &mut` soit par `transfert d'ownnership`, nous allons voir ces différents cas de figure.

> Les closures Rust se nomment `lambda` dans d'autres langages de programmation.

## Introduction

- Prenons le prototype de la methode for_each() du trait Iterator :

![FOREACH](pictures/for_each.png)

### Utilisation d'une fonction

- D'après son prototype, il est possible de donner une fonction à la méthode for_each (meme si dans la documentation ils disent "Calls a closure" ...). Le trait `fnMut` signifie que la fonction prend une référence mutable de Self::Item soit le type élément de notre Iterateur :

In [3]:
fn mul_by_2(inner: &mut u8) {
    *inner *= 2;
}

let mut v: Vec<u8> = vec!(1, 2, 4, 8);
v.iter_mut().for_each(mul_by_2);
dbg!(&v);

[src/lib.rs:110] &v = [
    2,
    4,
    8,
    16,
]


### Utilisation d'une closure

- Cependant, à l'usage, les développeurs Rust utiliseront a la place une closure ici : 

In [43]:
{
    let mul_by_2 = |inner: &mut u8| { // CECI EST UNE CLOSURE
        *inner *= 2;
    };
    let mut v: Vec<u8> = vec!(1, 2, 4, 8);
    v.iter_mut().for_each(mul_by_2);
    dbg!(&v);
}

[src/lib.rs:148] &v = [
    2,
    4,
    8,
    16,
]


()

> Mais quel avantage par rapport à la fonction précédente ?

- Dans ce cas précis, pas grand-chose. Mais la closure peut devenir intéressante si on cherche a **capturer une variable depuis l'extérieur de la closure**. En voici un exemple :

In [25]:
{
    let number = 3; // number sera capture
    let mul_by_number = |inner: &mut u8| { // CECI EST UNE CLOSURE
        *inner *= number;
    };
    let mut v: Vec<u8> = vec!(1, 2, 4, 8);
    v.iter_mut().for_each(mul_by_number);
    dbg!(&v);
    dbg!(number);
}

[src/lib.rs:133] &v = [
    3,
    6,
    12,
    24,
]
[src/lib.rs:134] number = 3


()

- Avec une fonction, cela aurait été impossible :

In [26]:
let number = 3;
fn mul_by_number(inner: &mut u8) {
    *inner *= number;
}

let mut v: Vec<u8> = vec!(1, 2, 4, 8);
v.iter_mut().for_each(mul_by_number);
dbg!(&v);

Error: cannot find value `number` in this scope

- La plupart du temps, les closures sont directement définies là où elles sont utilisées :

In [24]:
let mut v: Vec<u8> = vec!(1, 2, 4, 8);
v.iter_mut().for_each(|inner| *inner *= 2);
dbg!(&v);

let number = 3;
let mut v: Vec<u8> = vec!(1, 2, 4, 8);
v.iter_mut().for_each(|inner| *inner *= number);
dbg!(&v);

[src/lib.rs:112] &v = [
    2,
    4,
    8,
    16,
]
[src/lib.rs:117] &v = [
    3,
    6,
    12,
    24,
]


## Capture par référence

- Dans l'exemple suivant, la string s est passée a la closure par référence :

In [27]:
fn main() {
    // Declaration de la variable s en dehors de la closure
    let s = "MaString".to_string();
    // Ici on definit la closure avant de l'utiliser
    let closure = |inner: &mut u8| {
        *inner *= 2;
        // Ici la variable s est capturee par reference &s
        println!("One other iteration: {}", s);
    };
    let mut v: Vec<u8> = vec!(1, 2, 4, 8);
    v.iter_mut().for_each(closure);
    // s est toujours accessible ici
    dbg!(&s);

    // Ici la closure est anonyme
    v.iter_mut().for_each(|inner| {
        *inner *= 2;
        // Ici la variable s est capturee par reference &s
        println!("One other iteration: {}", s);
    });
    // s est toujours accessible ici
    dbg!(&s);
}
main();

One other iteration: MaString
One other iteration: MaString
One other iteration: MaString
One other iteration: MaString
One other iteration: MaString
One other iteration: MaString
One other iteration: MaString
One other iteration: MaString


[src/lib.rs:18] &s = "MaString"
[src/lib.rs:27] &s = "MaString"


> L'ownership de la String n'a pas été transféré a la closure puisque s est toujours accessible à la fin de la fonction. On dit que la closure a emprunte une référence de s.

**Ici, une référence non-mutable de s est passée implicitement a la closure, `il n'y a guère de différence de code si cela avait été une référence mutable`.**

## Transfert d'ownership

- Il existe un moyen de forcer le transfert de la Sting s a la closure :

In [28]:
fn main() {
    // Declaration de la variable s en dehors de la closure
    let s = "MaString".to_string();
    // Ici on definit la closure avec MOVE avant de l'utiliser
    let closure = move |inner: &mut u8| {
        *inner *= 2;
        // Ici, l'ownership de la variable s est passe au thread
        println!("One other iteration: {}", s);
    };
    let mut v: Vec<u8> = vec!(1, 2, 4, 8);
    v.iter_mut().for_each(closure);
    // s n'est plus accessible ici, le programme ne compile donc pas !
    // dbg!(s);
}
main()

One other iteration: MaString
One other iteration: MaString
One other iteration: MaString
One other iteration: MaString


()

> Le mot clef **move** force les variables a être capturees par la closure.

*C'est le cas de **thread**. Il serait, en effet, assez illogique étant donné les garanties de sécurité du langage que prodigue Rust qu'un thread puisse recevoir une référence d'une variable extérieure a celui-ci, si cette dernière réfère sur une variable présente sur la pile du thread appelant et non sur le tas, il est possible qu'au moment où le nouveau thread tente d'y accéder, la pile de la fonction appelante soit modifiée.*

![THREAD](pictures/thread.png)

In [38]:
let a: u32 = 12;
std::thread::spawn(|| { // LE COMPILATEUR PENSE POUVOIR PASSER LA VARIABLE PAR REFERENCE 
    dbg!(a); // J'accede a la valeur de a depuis mon thread.
});
dbg!(a); // Je veux continuer de pouvoir lire ma variable dans le thread principal.

Error: closure may outlive the current function, but it borrows `a`, which is owned by the current function

- Apres correction :

In [39]:
let a: u32 = 12;
std::thread::spawn(move || { // FORCE LE TRANSFERT DE L'OWNERSHIP DE MA VARIABLE VERS LE THREAD
    dbg!(a); // J'accede a la valeur de a depuis mon thread.
});

[src/lib.rs:128] a = 12


## Exercices

### Le code suivant ne compile pas, quelle explication donner ? Comment peut-on le corriger ?

In [57]:
{
    let mut number = 1;
    let mut v: Vec<u8> = vec!(1, 2, 4, 8);
    let ma_closure = |inner: &mut u8| {
        number *= 2;
        *inner *= number
    };
    dbg!(number);
    v.iter_mut().for_each(ma_closure);
    dbg!(&v);
    dbg!(number);
    number *= 2;
    dbg!(number);
}

Error: cannot use `number` because it was mutably borrowed

> Ici la variable number est passe a la closure par **reference mutable** puisque dans la closure, la variable number est modifiée. Déjà, si l'on n'avait pas modifié number dans la closure, le passage par référence aurait ete non-mutable et le programme compilerait, mais ce n'est pas la correction qu'il faut appliquer puisque l'on désire que number soit modifie une fois dans la closure...

> Le problème avec l'emprunt d'une référence mutable par la closure tient aux mécanismes que nous avons étudié plus tôt quant au comportement des références. La règle est : si j'emprunte une référence mutable, **je ne peux accéder à la variable autrement par cette meme reference**.

> Hors ligne 8, l'instruction dbg!(number) tente d'accéder à cette variable. Ce qui est interdit. Pour corriger le code, il suffit de supprimer la ligne 8.